In [5]:
import os
from langchain.memory import  ConversationBufferMemory
from langchain.memory import  ConversationBufferWindowMemory

api_key = os.getenv("OPENAI_API_KEY")

In [3]:
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("我是人类")
memory.chat_memory.add_ai_message("我是AI，有什么可以帮到你？")
memory.load_memory_variables({})

C:\Users\25746\AppData\Local\Temp\ipykernel_13596\2245175809.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


{'history': 'Human: 我是人类\nAI: 我是AI，有什么可以帮到你？'}

In [9]:
memory = ConversationBufferWindowMemory(k=1)

memory.save_context({"input":"你好，我是人类!"},{"output":"你好，我是AI，有什么可以帮助你的吗？"})
memory.save_context({"input":"我想吃鸡肉"},{"output":"好的，我帮你找找鸡肉的做法"})

memory.load_memory_variables({})

{'history': 'Human: 我想吃鸡肉\nAI: 好的，我帮你找找鸡肉的做法'}

In [11]:
from langchain.llms import  OpenAI
from langchain.memory import ConversationEntityMemory

In [19]:
llm = OpenAI(
    temperature=0
)

memory = ConversationEntityMemory(llm = llm)
_input = {
    "input": "胡八一和王胖子雪莉杨经常在一起冒险，合称盗墓铁三角"
}
memory.load_memory_variables(_input)
memory.save_context(
    _input,
    {
        "output":"听起来很刺激，我也想加入他们！"
    }
)
memory.load_memory_variables({"input":"铁三角是谁?"})

{'history': 'Human: 胡八一和王胖子雪莉杨经常在一起冒险，合称盗墓铁三角\nAI: 听起来很刺激，我也想加入他们！',
 'entities': {}}

In [21]:
from langchain.memory import ConversationKGMemory

In [23]:
memory = ConversationKGMemory(llm = llm)

memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)

memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)

In [25]:
memory.load_memory_variables({"input":"tomie是谁?"})

{'history': 'On tomie: tomie is a training instructor.'}

In [27]:
memory.get_current_entities("tomie最喜欢做什么事?")

['tomie']

In [29]:
memory.get_knowledge_triplets("tomie最喜欢打游戏")

[KnowledgeTriple(subject='tomie', predicate='最喜欢', object_='打游戏')]

In [31]:
from langchain.memory import ConversationSummaryMemory

In [33]:
memory = ConversationSummaryMemory(llm=llm)
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)

memory.load_memory_variables({})

C:\Users\25746\AppData\Local\Temp\ipykernel_13596\2633398412.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm)


{'history': '\nThe human asks the AI to help find something called "tomie". The AI apologizes and asks what "tomie" is. The human explains that "tomie" is a training instructor. The AI acknowledges and says it understands.'}

In [35]:
messages = memory.chat_memory.messages 
#print(messages)
memory.predict_new_summary(messages,"")

'\nThe human asks the AI to help find information about someone named Tomie. The AI asks for clarification on who Tomie is, and the human explains that Tomie is a training instructor. The AI acknowledges this and says it understands.'

In [39]:
from langchain.memory import ConversationSummaryBufferMemory

In [41]:

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=10,
    return_messages=True
)
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)
memory.save_context(
    {"input":"今天他要讲一门关于RAG的课程"},
    {"output":"好的，我知道了。需要RAG的资料吗？"}
)


C:\Users\25746\AppData\Local\Temp\ipykernel_13596\2164936767.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


In [43]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='\nThe human asks the AI to help find something called "tomie". The AI apologizes and asks what "tomie" is. The human explains that "tomie" is a training instructor. The AI acknowledges and says it understands. The human then tells the AI that the instructor will be teaching a course on RAG today. The AI confirms and asks if it needs any materials on RAG.', additional_kwargs={}, response_metadata={})]}

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS

In [ ]:
memory = ConversationBufferMemory()
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)
memory.save_context(
    {"input":"今天他要讲一门关于RAG的课程"},
    {"output":"好的，我知道了。需要RAG的资料吗？"}
)
memory.save_context(
    {"input":"不需要资料了，谢谢"},
    {"output":"好的，那我就不打扰你了。"}
)

vectorstore = FAISS.from_texts(
    memory.buffer.split("\n"),
    OpenAIEmbeddings()
)
FAISS.save_local(vectorstore,"test_faiss")

In [ ]:
FAISS.load_local("test_faiss",OpenAIEmbeddings(model="text-embedding-3-small"),allow_dangerous_deserialization=True).similarity_search("tomie是什么职业?")

In [ ]:
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.memory import VectorStoreRetrieverMemory
r1 = FAISS.load_local("test_faiss",OpenAIEmbeddings(),allow_dangerous_deserialization=True)
r2 = r1.as_retriever(
    search_kwargs={"k":1}
)
memory2 = VectorStoreRetrieverMemory(
    retriever=r2
)
memory2.load_memory_variables({"prompt":"tomie是什么职业?"})